# An introductory showcase  

## A quick overview of some useful high level python functionality that is covered in the course content

The example specifically showcases some of python's usefulness in opening, handling, viewing and plotting data, using a sea ice concentration dataset as a test case. 

It is not necessary (or expected) for this to be understood yet -- the idea at this stage is to give some context to the course content coming up, and demonstrate how a lot can be achieved using a short and intuitive commands. I have added comments for completeness, but these concepts will make more sense in the individual lessions. In addition to the data handling described above, the example also relies on various commonly-used python features that will be covered in the course, such as:
- The use of python packages, and their complementarity to each other
- The object-oriented nature of python
- Indexing in python
- the use of text strings and their seamless use in indexing

### Load in python packages

These each handle different things, such as reading and handling data (xarray; numpy), and plotting data (matplotlib; cartopy). These open-source packages are often developed to complement each other. 

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np

### Open a sea-ice dataset for September 1999

Read a netcdf dataset using the xarray package. To do this we first call the package we want to use (in this case xarray, shortened to xr when we imported it above), then use its open_dataset functionality for reading the datafile (specified after a full stop. I.e. xr.open_dataset('/text/string/to/file/location'). 

These datasets can contain many variables (in this case, 8 different sea ice variables). The data is displayed below, and can be interacted with by clicking on the icons for more information about their data contents (e.g. variables and their coordinates) and their attributes/metadata (e.g. names, sources,  units, creators...). 

In [ ]:
# sea ice variables for September 1999
seaice_data_monthly_199109 = xr.open_dataset('../data/seaice_conc_monthly_nh_f08_199109_v02r00.nc')
seaice_data_monthly_199109

### Identify and plot a variable using its name 

Select the seaice_conc_monthly_cdr variable, showing monthly sea ice concentration for that September 1999. Notice that it handles text easily, meaning we can directly use the name of the variable rather than a numbered index location. 

In [ ]:
seaice_conc_199109 = seaice_data_monthly_199109['seaice_conc_monthly_cdr']

xarray has a lot of functionality, including methods for data manipulation and for directly plotting the data. 

In [ ]:
# Use xarray's .plot() functionality to directly plot the data. 
# It automatically makes a 2D plot when given a 2D variable. It also automatically adds the names of variables, title & coordinates from the metadata
seaice_conc_199109.plot() 

### Now lets read in sea ice from a different year, for comparison. September 2012:

In [ ]:
seaice_data_monthly_201209 = xr.open_dataset('../data/seaice_conc_monthly_nh_f17_201209_v02r00.nc')
# And extract the sea ice concentration variable from it:
seaice_conc_201209 = seaice_data_monthly_201209['seaice_conc_monthly_cdr']

In [ ]:
# Now plot the change in sea ice extent
plt.subplots(figsize=(5, 4))  # creates a figure of a particular size.
sea_ice_change = (seaice_conc_201209.isel(time = 0) - seaice_conc_199109.isel(time = 0))
sea_ice_change.plot()

Now improve the projection using (lon, lat) on a polar projection instead of distances, and plot land. Uses the cartopy (ccrs) package, which complements the plotting. 
Notice that the ax.coastlines and ax.gridlines functions are called from directly from ax. It can do this because ax was created by the matplotlib (plt) package and so python knows that it is a matplotlib 'object' that comes with a set of callable functions. 

In [ ]:
new_projection = ccrs.NorthPolarStereo()  # set the cartopy (ccrs) NorthPolarStereo projection to new_projection
fig, ax = plt.subplots(subplot_kw={'projection': new_projection}, figsize=(5, 4)) #  fig and ax are variables for the figure properties and individual axis properties. Used to specify features, such as the coastlines 
sea_ice_change.plot(x='longitude',y='latitude',transform=ccrs.PlateCarree()) # specify the 'longitude' and 'latitude' coordinates for the x and y dimensions. PlateCarree is the projection the data is on, which gets transformed to the new_projection
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.title('Sea Ice conc change, Sept 1999 - Sept 2012');

## Now use python 'lists' to help us plot all variables in the dataset. 

This can save us having to type out all of the variables and their coordinates etc every time we want to open a dataset. For example, if we are reading in many datasets that we are unfamiliar with, we can use python's ability to handle text strings to output all variables in a dataset and plot them one by one, in each case automatically displaying their variable names and coordinates etc.  

In [ ]:
# Create a list of all variables in the dataset. 
variable_list = seaice_data_monthly_201209.keys()

In [ ]:
# Show these as a list
list(variable_list)

In [ ]:
# Plot all variables using a for loop over the variable list. 
Num_variables = len(seaice_data_monthly_201209)
fig, ax = plt.subplots(nrows=round(Num_variables/2),ncols=2,figsize=(15,25)) # specify the number of columns and rows according to the number of variables
ax=ax.flatten()         # A trick to convert the matrix of axes (e.g. with dimensions (5,2) to a sequential list of (10,1)). 
i=0                     # start with the first set of plot axes (i.e. ax[0])
for key in variable_list:
    if seaice_data_monthly_201209[key].coords: # this checks that the variable contains coordinates. If $seaice_data_monthly_201209[key].coords is empty, it fails the condition. 
        seaice_data_monthly_201209[key].plot(ax=ax[i])
        i=i+1  # increase i to plot on the next set of plot axes (i.e. ax[i+1])